In [1]:
!pip install git+https://github.com/Farama-Foundation/MAgent2

  Cloning https://github.com/Farama-Foundation/MAgent2 to /tmp/pip-req-build-b6af_q0_
  Running command git clone --filter=blob:none --quiet https://github.com/Farama-Foundation/MAgent2 /tmp/pip-req-build-b6af_q0_
  Resolved https://github.com/Farama-Foundation/MAgent2 to commit b2ddd49445368cf85d4d4e1edcddae2e28aa1406
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 99.4 MB/s eta 0:00:00:00:01:01
  Created wheel for magent2: filename=magent2-0.3.3-cp310-cp310-linux_x86_64.whl size=1696136 sha256=d3ca8842de76a311865050b8026c00c8fd92a9e885c983b1d9f19ced8912a886
  Stored in directory: /tmp/pip-ephem-wheel-cache-5j_vl7xh/wheels/e4/8e/bf/51a30bc4038546e23b81c9fb513fe6a8fd916e5a9c5f4291d5
Successfully built magent2


In [2]:
!git clone https://github.com/giangbang/RL-final-project-AIT-3007.git

Cloning into 'RL-final-project-AIT-3007'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 56 (delta 12), reused 13 (delta 6), pack-reused 32 (from 1)
Receiving objects: 100% (56/56), 17.78 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [3]:
import sys
sys.path.append('/kaggle/working/RL-final-project-AIT-3007')

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
from collections import deque, Counter
import os
from magent2.environments import battle_v4
import time
# from torch_model import QNetwork

class MyQNetwork(nn.Module):
    def __init__(self, observation_shape, action_shape):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(observation_shape[-1], 13, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(13, 13, kernel_size=3),
            nn.ReLU()
        )
        dummy_input = torch.randn(observation_shape).permute(2, 0, 1)
        dummy_output = self.cnn(dummy_input)
        flatten_dim = dummy_output.view(-1).shape[0]
        self.fc = nn.Sequential(
            nn.Linear(flatten_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, action_shape)
        )

    def forward(self, x):
        assert len(x.shape) >= 3, "only support magent input observation"
        x = self.cnn(x)
        if len(x.shape) == 3:
            batchsize = 1
        else:
            batchsize = x.shape[0]
        x = x.reshape(batchsize, -1)
        return self.fc(x)

class ReplayBuffer(Dataset):
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = deque(maxlen=capacity)
        
    def add(self, state, action, reward, next_state, done):
        experience = (state, action, reward, next_state, done)
        self.buffer.append(experience)

    def __len__(self):
        return len(self.buffer)

    def __getitem__(self, index):
        return self.buffer[index]

In [5]:
class Trainer:
    def __init__(self, env, input_shape, action_shape, learning_rate=1e-3):
        self.env = env
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.q_network = MyQNetwork(input_shape, action_shape).to(self.device)
        self.target_network = MyQNetwork(input_shape, action_shape).to(self.device)
        self.target_network.load_state_dict(self.q_network.state_dict())

        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.replay_buffer = ReplayBuffer(capacity=16200 * 10)

        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9
        self.update_target_every = 2

    def select_action(self, observation, agent):
        if np.random.rand() <= self.epsilon:
            return self.env.action_space(agent).sample()

        observation = (
            torch.FloatTensor(observation).unsqueeze(0).to(self.device)
        )
        with torch.inference_mode():
            q_values = self.q_network(observation)
        return torch.argmax(q_values, dim=1).item()

    def training(self, episodes=50, batch_size=2 ** 10):        
        for episode in range(episodes):
            self.env.reset()
            
            total_reward = 0
            reward_for_agent = {agent: 0 for agent in self.env.agents if agent.startswith('blue')}
            prev_observation = {}
            prev_action = {}
            self.env.reset()
            step = 0

            for idx, agent in enumerate(self.env.agent_iter()):
                step += 1
                observation, reward, termination, truncation, info = self.env.last()
                observation = np.transpose(observation, (2, 0, 1))
                
                agent_handle = agent.split('_')[0]
                
                if agent_handle == 'blue':
                    total_reward += reward
                    reward_for_agent[agent] += reward
                    
                if termination or truncation:
                    action = None
                else:
                    if agent_handle == 'blue':
                        action = self.select_action(observation, agent)
                    else:
                        action = self.env.action_space(agent).sample()

                if agent_handle == 'blue':
                    prev_observation[agent] = observation
                    prev_action[agent] = action
                
                self.env.step(action)
                
                if (idx + 1) % self.env.num_agents == 0:
                    break
                
            for agent in self.env.agent_iter():
                step += 1
                
                observation, reward, termination, truncation, info = self.env.last()
                observation = np.transpose(observation, (2, 0, 1))
                
                agent_handle = agent.split('_')[0]
                
                if agent_handle == 'blue':
                    total_reward += reward
                    reward_for_agent[agent] += reward
                    
                if termination or truncation:
                    action = None
                else:
                    if agent_handle == 'blue':
                        action = self.select_action(observation, agent)
                    else:
                        action = self.env.action_space(agent).sample()
    
                    if agent_handle == 'blue':
                        self.replay_buffer.add(
                            prev_observation[agent],
                            prev_action[agent],
                            reward,  
                            observation,
                            termination
                        )

                        prev_observation[agent] = observation
                        prev_action[agent] = action
    
                self.env.step(action)
            
            dataloader = DataLoader(self.replay_buffer, batch_size=batch_size, shuffle=True)
            self.update_model(dataloader)
                
            if (episode + 1) % self.update_target_every == 0:
                self.target_network.load_state_dict(self.q_network.state_dict())
    
            max_reward = max(reward_for_agent.values())
            
            print(f"Episode {episode}, Epsilon: {self.epsilon:.2f}, Total Reward: {total_reward}, Steps: {step}, Max Reward: {max_reward} ")
            self.epsilon = max(self.epsilon * self.epsilon_decay, self.epsilon_min)

    def update_model(self, dataloader):
        for states, actions, rewards, next_states, dones in dataloader:

            states = torch.tensor(states, dtype=torch.float32).to(self.device)
            actions = torch.tensor(actions, dtype=torch.long).to(self.device)
            rewards = torch.tensor(rewards, dtype=torch.float32).to(self.device)
            next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
            dones = torch.tensor(dones, dtype=torch.float32).to(self.device)

            current_q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
            with torch.inference_mode():
                next_q_values = self.target_network(next_states).max(1)[0]
            expected_q_values = rewards + (self.gamma * next_q_values * (1 - dones))

            loss = self.criterion(current_q_values, expected_q_values)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

In [6]:
env = battle_v4.env(map_size=45, render_mode=None)
trainer = Trainer(env, env.observation_space("red_0").shape, env.action_space("red_0").n)
trainer.training()

/tmp/ipykernel_30/2370074755.py:118: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states = torch.tensor(states, dtype=torch.float32).to(self.device)
/tmp/ipykernel_30/2370074755.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  actions = torch.tensor(actions, dtype=torch.long).to(self.device)
/tmp/ipykernel_30/2370074755.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rewards = torch.tensor(rewards, dtype=torch.float32).to(self.device)
/tmp/ipykernel_30/2370074755.py:121: UserWarning: To copy construct from a tensor, it is reco

Episode 0, Epsilon: 1.00, Total Reward: -3266.5901186224073, Steps: 161184, Max Reward: -7.990000298246741 
Episode 1, Epsilon: 0.90, Total Reward: -2944.3601070651785, Steps: 158089, Max Reward: -9.960000389255583 
Episode 2, Epsilon: 0.81, Total Reward: -2372.090096999891, Steps: 116736, Max Reward: -4.465000241063535 
Episode 3, Epsilon: 0.73, Total Reward: -2343.700088308193, Steps: 126187, Max Reward: -0.20000114664435387 
Episode 4, Epsilon: 0.66, Total Reward: -2223.100078023039, Steps: 140005, Max Reward: -17.500000898726285 
Episode 5, Epsilon: 0.59, Total Reward: -1884.2900680135936, Steps: 122853, Max Reward: -1.7700000759214163 
Episode 6, Epsilon: 0.53, Total Reward: -1874.3000619122759, Steps: 150544, Max Reward: -4.600000815466046 
Episode 7, Epsilon: 0.48, Total Reward: -1766.3000552961603, Steps: 155133, Max Reward: 15.399999102577567 
Episode 8, Epsilon: 0.43, Total Reward: -1295.400049522519, Steps: 91150, Max Reward: 23.39999925531447 
Episode 9, Epsilon: 0.39, Tota

In [7]:
os.makedirs("models", exist_ok=True)
torch.save(trainer.q_network.state_dict(), "models/blue_vs_random.pt")
print("Training complete. Model saved.")

Training complete. Model saved.


In [9]:
# make video
import cv2

env = battle_v4.env(map_size=45, render_mode="rgb_array", max_cycles=300)
vid_dir = "video"
os.makedirs(vid_dir, exist_ok=True)
fps = 35
frames = []
my_q_network = MyQNetwork(
    env.observation_space("red_0").shape, env.action_space("red_0").n
)
my_q_network.load_state_dict(
    torch.load("/kaggle/working/models/blue_vs_random.pt", weights_only=True, map_location="cpu")
)
my_q_network.eval()

env.reset()
for agent in env.agent_iter():

    observation, reward, termination, truncation, info = env.last()
    observation = (
        torch.Tensor(observation).float().permute([2, 0, 1]).unsqueeze(0)
    )
    if termination or truncation:
        action = None  # this agent has died
    else:
        agent_handle = agent.split("_")[0]
        if agent_handle == "blue":
            with torch.inference_mode():
                q_values = my_q_network(observation)
            action = torch.argmax(q_values, dim=1).numpy()[0]
        else:
            action = env.action_space(agent).sample()
            
    env.step(action)

    if agent == "red_0":
        frames.append(env.render())

height, width, _ = frames[0].shape
out = cv2.VideoWriter(
    os.path.join(vid_dir, f"blue_vs_random.mp4"),
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (width, height),
)
for frame in frames:
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    out.write(frame_bgr)
out.release()
print("Done recording My blue agents vs random")

env.close()

Done recording My blue agents vs random


In [10]:
from torch_model import QNetwork as Q_pretrain
from final_torch_model import QNetwork as Q_final

try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x, *args, **kwargs: x  # Fallback: tqdm becomes a no-op
    
def eval():
    max_cycles = 300
    env = battle_v4.env(map_size=45, max_cycles=max_cycles)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    def random_policy(env, agent, obs):
        return env.action_space(agent).sample()

    MyNetwork = MyQNetwork(
        env.observation_space("red_0").shape, env.action_space("red_0").n
    )
    MyNetwork.load_state_dict(
        torch.load("/kaggle/input/blue-improved/blue_improved.pt", weights_only=True, map_location='cpu')
    )
    MyNetwork.to(device)

    red_pretrained = Q_pretrain(
        env.observation_space("red_0").shape, env.action_space("red_0").n
    )
    red_pretrained.load_state_dict(
        torch.load("/kaggle/working/RL-final-project-AIT-3007/red.pt", weights_only=True, map_location="cpu")
    )
    red_pretrained.to(device)

    red_final = Q_final(
        env.observation_space("red_0").shape, env.action_space("red_0").n
    )
    red_final.load_state_dict(
        torch.load('/kaggle/working/RL-final-project-AIT-3007/red_final.pt', weights_only=True, map_location='cpu')
    )
    red_final.to(device)
    
    v1 = Q_pretrain(
        env.observation_space("red_0").shape, env.action_space("red_0").n
    )
    v1.load_state_dict(
        torch.load("/kaggle/input/another-model/v1.pth", weights_only=True, map_location="cpu")
    )
    v1.to(device)

    v2 = Q_pretrain(
        env.observation_space("red_0").shape, env.action_space("red_0").n
    )
    v2.load_state_dict(
        torch.load("/kaggle/input/another-model/v2.pth", weights_only=True, map_location="cpu")
    )
    v2.to(device)

    def pretrain_policy(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        with torch.inference_mode():
            q_values = red_pretrained(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]

    def final_policy(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        with torch.inference_mode():
            q_values = red_final(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]
        
    def v1_policy(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        with torch.inference_mode():
            q_values = v1(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]

    def v2_policy(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        with torch.inference_mode():
            q_values = v2(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]

    def my_policy(env, agent, obs):
        observation = (
            torch.Tensor(obs).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        with torch.inference_mode():
            q_values = MyNetwork(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]

    def run_eval(env, red_policy, blue_policy, n_episode: int = 100):
        red_win, blue_win = [], []
        red_tot_rw, blue_tot_rw = [], []
        n_agent_each_team = len(env.env.action_spaces) // 2

        for _ in tqdm(range(n_episode)):
            env.reset()
            n_kill = {"red": 0, "blue": 0}
            red_reward, blue_reward = 0, 0

            for agent in env.agent_iter():
                observation, reward, termination, truncation, info = env.last()
                agent_team = agent.split("_")[0]

                n_kill[agent_team] += (
                    reward > 4.5
                )  # This assumes default reward settups
                if agent_team == "red":
                    red_reward += reward
                else:
                    blue_reward += reward

                if termination or truncation:
                    action = None  # this agent has died
                else:
                    if agent_team == "red":
                        action = red_policy(env, agent, observation)
                    else:
                        action = blue_policy(env, agent, observation)

                env.step(action)

            who_wins = "red" if n_kill["red"] >= n_kill["blue"] + 5 else "draw"
            who_wins = "blue" if n_kill["red"] + 5 <= n_kill["blue"] else who_wins
            red_win.append(who_wins == "red")
            blue_win.append(who_wins == "blue")

            red_tot_rw.append(red_reward / n_agent_each_team)
            blue_tot_rw.append(blue_reward / n_agent_each_team)

        return {
            "winrate_red": np.mean(red_win),
            "winrate_blue": np.mean(blue_win),
            "average_rewards_red": np.mean(red_tot_rw),
            "average_rewards_blue": np.mean(blue_tot_rw),
        }

    print("=" * 20)
    print("Eval with pretrain policy")
    print(
        run_eval(
            env=env, red_policy=pretrain_policy, blue_policy=my_policy, n_episode=30
        )
    )
    print("=" * 20)
    print("Eval with final policy")
    print(
        run_eval(
            env=env, red_policy=final_policy, blue_policy=my_policy, n_episode=30
        )
    )

    print("=" * 20)
    print("Eval with v1 policy")
    print(
        run_eval(
            env=env, red_policy=v1_policy, blue_policy=my_policy, n_episode=30
        )
    )
    print("=" * 20)

    print("Eval with v2 policy")
    print(
        run_eval(
            env=env, red_policy=v2_policy, blue_policy=my_policy, n_episode=30
        )
    )
    print("=" * 20)

if __name__ == "__main__":
    eval()

/opt/conda/lib/python3.10/site-packages/pettingzoo/utils/wrappers/base.py:72: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(


Eval with pretrain policy


100%|██████████| 30/30 [01:40<00:00,  3.36s/it]


{'winrate_red': 0.0, 'winrate_blue': 1.0, 'average_rewards_red': 0.7385699528127663, 'average_rewards_blue': 4.133952614933313}
Eval with final policy


100%|██████████| 30/30 [03:56<00:00,  7.89s/it]


{'winrate_red': 0.0, 'winrate_blue': 1.0, 'average_rewards_red': 1.7948497787010647, 'average_rewards_blue': 0.3590224500645889}
Eval with v1 policy


100%|██████████| 30/30 [03:00<00:00,  6.03s/it]


{'winrate_red': 0.06666666666666667, 'winrate_blue': 0.8666666666666667, 'average_rewards_red': 3.799790105453612, 'average_rewards_blue': 2.884337363726701}
Eval with v2 policy


100%|██████████| 30/30 [02:03<00:00,  4.12s/it]

{'winrate_red': 0.6333333333333333, 'winrate_blue': 0.16666666666666666, 'average_rewards_red': 4.637506149958727, 'average_rewards_blue': 3.3422694900161267}
